### Quick start : Run a RNA task easily

First, generate the necessary index files

You can use the following command: 
```
$ rnaglib_index
```

Choose the task appropriate to your model. Here, we chose _RNA-Site_, a task instance called `LigandBindindSite` for illustration.
When instantiating the task, custom splitters or other arguments can be passed if needed.

In [1]:
from rnaglib.tasks import BindingSiteDetection
from rnaglib.transforms import FeaturesComputer, GraphRepresentation

task = BindingSiteDetection(root='tutorial') # You can pass arguments to use a custom splitter or dataset etc. if desired.

Index file not found at /Users/wissam/.rnaglib/indexes/rnaglib-nr-1.0.0.json. Run rnaglib_index
>>> Loading splits...
>>> Done


/opt/anaconda3/envs/RNApython/lib/python3.11/site-packages/networkx/readwrite/json_graph/node_link.py:287: FutureWarning: 
The default value will be changed to `edges="edges" in NetworkX 3.6.

To make this warning go away, explicitly set the edges kwarg, e.g.:

  nx.node_link_graph(data, edges="links") to preserve current behavior, or
  nx.node_link_graph(data, edges="edges") for forward compatibility.
  warnings.warn(


Choose your features and your targets (here, the feature is the nucleotide code and the target is the binding site)

In [2]:
from rnaglib.encoders import BoolEncoder
from rnaglib.transforms import FeaturesComputer
task.dataset.features_computer = FeaturesComputer(nt_features=['nt_code'], nt_targets='binding_site', custom_encoders= {'binding_site' : BoolEncoder()})

Choose your representation (here Pytorch Geometric graphs)

In [3]:
task.dataset.add_representation(GraphRepresentation(framework='pyg'))

>>> Adding <rnaglib.transforms.represent.graph.GraphRepresentation object at 0x177197510> representations.


Set train, validation and test loaders

In [4]:
task.set_loaders()

Define your model and training (here, a dummy model)

In [5]:
model = task.dummy_model

Evaluate the model on the defined task

In [6]:
final_result = task.evaluate(model)
print(final_result)

{'accuracy': 0.4894837476099426, 'mcc': -0.015326770874804748, 'f1': 0.47337278106508873}


### Get familiar with RNAglib objects

#### RNADataset

RNADataset objects represent a set of RNAs, each one being represented by its 3D structure.

In facts, each item of the RNADataset is encoded by a dictionary containing the networkx Graph representing the RNA.

It is also possible to add Representation and FeaturesComputer objects to a RNADataset.

To create a default RNA Dataset, you can run the code below

In [8]:
from rnaglib.data_loading import RNADataset

dataset = RNADataset()

Database was found and not overwritten


#### Transform

The Transform class groups all the objects which maps the dictionaries representing RNAs (i.e. an item of a RNADataset object) into another object.

A specific tutorial gives further details about this class: https://rnaglib.org/en/latest/rnaglib.transforms.html

Below are detailed two particular types of Transforms: Representations and FeaturesComputers.

##### Representation

A Representation object is a Tranaform that maps a RNA dictionary (as defined above) into a mathematical representation of this RNA. In the current version of RNAGlib, 4 representations are already implemented: GraphRepresentation, PointCloudRepresentation, VoxelRepresentation and Rings

GraphRepresentation converts RNA into a Leontis-Westhof graph (2.5D) where nodes are residues and edges are either base pairs or backbones.

PointCloudRepresentation converts RNA into a 3D point cloud based representation.

VoxelRepresentation converts RNA into a voxel/3D grid representation.

##### FeaturesComputer

A FeaturesComputer is a Transform that maps a RNA Dictionary into a dictionary of features (both RNA-level and node-level features).

### Advanced functionalities

#### Adding a representation to a RNADataset

A Representation object (or several representations grouped in a list) can be passed as argument to RNADataset during its creation or added to a RNADataset which has already been created using the following code:

In [9]:
from rnaglib.transforms import GraphRepresentation, PointCloudRepresentation
representations_list = [GraphRepresentation, PointCloudRepresentation]
dataset.add_representation(representations_list)

>>> Adding [<class 'rnaglib.transforms.represent.graph.GraphRepresentation'>, <class 'rnaglib.transforms.represent.point_cloud.PointCloudRepresentation'>] representations.


#### Removing a representation

It is possible to remove one or several Representation object(s) from a RNADataset using the code below by using their names (it could be "graph" for GraphRepresentation, "point_cloud" for PointCloudRepresentation, "voxel" for VoxelRepresentation or "ring" for RingRepresentation)

In [16]:
dataset.remove_representation(["point_cloud"])

#### Adding features to a RNADataset



### Customization

#### Create a custom Transform